In [1]:
!pip install PyPDF2 transformers nltk gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 342.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/1

In [3]:
import gradio as gr
from google.colab import files
import PyPDF2
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from transformers import pipeline
import re

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Function to extract text from PDF
def extract_text_from_pdf(pdf_file):
    reader = PyPDF2.PdfReader(pdf_file)
    text = ""
    for page_num in range(len(reader.pages)):
        text += reader.pages[page_num].extract_text()
    return text

# Function to preprocess text
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    cleaned_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    return ' '.join(cleaned_tokens)

# Load the QA pipeline model
qa_pipeline = pipeline("question-answering")

# Function to answer a question using the QA model
def answer_question(question, context):
    result = qa_pipeline(question=question, context=context)
    return result['answer']

# Function to split the report into sections
def split_report_into_sections(text):
    sections = re.split(r'\n\s*\n', text)  # Split based on double newline
    section_dict = {}
    for section in sections:
        if '\n' in section:
            heading, content = section.split('\n', 1)
            section_dict[heading.lower()] = content
    return section_dict

# Function to get the relevant section based on the question
def get_relevant_section(question, section_dict):
    for heading in section_dict:
        if any(word in heading for word in question.lower().split()):
            return section_dict[heading]
    return " ".join(section_dict.values())  # If no match, search the whole report

# Main function with sections
def main_with_sections(pdf_file, question):
    # Step 1: Extract and split the report into sections
    text = extract_text_from_pdf(pdf_file)
    section_dict = split_report_into_sections(text)

    # Step 2: Get the relevant section based on the question
    relevant_section = get_relevant_section(question, section_dict)

    # Step 3: Answer the question using the QA model
    answer = answer_question(question, relevant_section)

    return answer

# Gradio Interface Function
def process_pdf_question(pdf_file, question):
    return main_with_sections(pdf_file.name, question)

# Creating Gradio Interface using the updated API
with gr.Blocks() as interface:
    gr.Markdown("# PDF Question Answering System")
    gr.Markdown("Upload a PDF file and ask a question. The system will analyze the text and provide an answer.")

    pdf_input = gr.File(label="Upload PDF")
    question_input = gr.Textbox(label="Ask a Question", placeholder="e.g., What is the company's profit after tax?")

    output_text = gr.Textbox(label="Answer")

    # When the button is clicked, the process_pdf_question function will be executed
    gr.Button("Submit").click(process_pdf_question, inputs=[pdf_input, question_input], outputs=output_text)

# Launch the interface
interface.launch()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5095783b384a21bd42.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
